<a href="https://colab.research.google.com/github/hbisgin/BigDatav1/blob/main/Lecture6_SQLDataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/databricks/LearningSparkV2/refs/heads/master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv

--2025-09-17 03:12:35--  https://raw.githubusercontent.com/databricks/LearningSparkV2/refs/heads/master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33396236 (32M) [text/plain]
Saving to: ‘departuredelays.csv’

departuredelays.csv 100%[===================>]  31.85M   201MB/s    in 0.2s    

2025-09-17 03:12:35 (201 MB/s) - ‘departuredelays.csv’ saved [33396236/33396236]



In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Example").enableHiveSupport(). getOrCreate()
#sc = spark.sparkContext #for backward compatibility, RDD would need this

In [7]:
#to be on the safe side I prepare the schema
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("date", StringType(), True),        # date as string!!!
    StructField("delay", IntegerType(), True),
    StructField("distance", IntegerType(), True),
    StructField("origin", StringType(), True),
    StructField("destination", StringType(), True)
])


In [8]:
df = spark.read.option("header", "true").schema(schema).csv("departuredelays.csv")
#I prepared schema to make sure date column is read as string type.

In [9]:
df.printSchema()
df.show(3)

root
 |-- date: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 3 rows



In [21]:
from pyspark.sql.functions import col, lpad, substring, concat_ws, to_date,to_timestamp, current_date, concat, year

# Pad to 7 digits: e.g., 1010005 -> "1010005"
#df = df.withColumn("date_str", lpad(col("date").cast("string"), 7, "0")) #add zero to the left
df = df.withColumn("as_time_stamp", to_timestamp(concat(year(current_date()), col("date")), "yyyyMMddHHmm") )








In [22]:
df.printSchema()
df.show(3)

root
 |-- date: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- as_time_stamp: timestamp (nullable = true)

+--------+-----+--------+------+-----------+-------------------+
|    date|delay|distance|origin|destination|      as_time_stamp|
+--------+-----+--------+------+-----------+-------------------+
|01011245|    6|     602|   ABE|        ATL|2025-01-01 12:45:00|
|01020600|   -8|     369|   ABE|        DTW|2025-01-02 06:00:00|
|01021245|   -2|     602|   ABE|        ATL|2025-01-02 12:45:00|
+--------+-----+--------+------+-----------+-------------------+
only showing top 3 rows



In [ ]:
df.createOrReplaceTempView("us_delay_flights_tbl")

In [ ]:
spark.sql("CREATE DATABASE my_spark_db")
spark.sql("USE my_spark_db")


DataFrame[]

In [ ]:
#spark.sql("CREATE TABLE dummy(date STRING, delay INT,distance INT, origin STRING, destination STRING)") #we may need HIVE setup for this

In [ ]:
df.write.saveAsTable("managed_us_delay_flights_tbl2") #managed


In [ ]:
df.write.option("path", "/tmp/data/us_flights_delay").saveAsTable("us_delay_flights_tbl_unmanaged") #unmanaged

In [ ]:
# Create another session
spark2 = spark.newSession()

# Create temp view in first session
df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
df.createOrReplaceTempView("people")

# Query in first session -> works
spark.sql("SELECT * FROM us_delay_flights_tbl_unmanaged where delay<0").show()

# Query in second session -> fails (temp view not shared)
#spark2.sql("SELECT * FROM us_delay_flights_tbl_unmanaged where delay<0").show() # Would error

# Each session has its own catalog of temp views
df2 = spark2.createDataFrame([(3, "Carol")], ["id", "name"])
df2.createOrReplaceTempView("people2")

# Accessible in spark2 but not in spark
spark2.sql("SELECT * FROM people2").show()

In [ ]:
print(spark.catalog.listDatabases())
print(spark.catalog.listTables())
spark.catalog.listColumns("us_delay_flights_tbl")

[Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='file:/content/spark-warehouse'), Database(name='learn_spark_db', catalog='spark_catalog', description='', locationUri='file:/content/spark-warehouse/learn_spark_db.db'), Database(name='my_spark_db', catalog='spark_catalog', description='', locationUri='file:/content/spark-warehouse/my_spark_db.db')]
[Table(name='people', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True), Table(name='us_delay_flights_tbl', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]


[Column(name='date', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='dateformatted', description=None, dataType='date', nullable=True, isPartition=False, isBucket=False)]

Can you get the m&m data from https://raw.githubusercontent.com/databricks/LearningSparkV2/refs/heads/master/chapter2/py/src/data/mnm_dataset.csv and perform the following:

1. Read it as a csv file
2. Register as a view
3. Select from the DataFrame the fields "State", "Color", and "Count" in a way that it should report the count of each color for each state. In other words, grouping by state and color pairs should be counted.
4. Report what if we just want to see the data for a single state, e.g., CA?
